In [46]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import earthaccess
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# veritable buffet for marine microbes

In [2]:
auth = earthaccess.login()

In [22]:
tspan = ("2024-12-10", "2025-01-30")
bbox = (-131.9569, 29.4430, -120.8539, 37.9144) # west, south, east, north


In [34]:
results = earthaccess.search_data(
    short_name="PACE_OCI_L2_BGC",
    temporal=tspan,
    bounding_box=bbox
)
len(results)

results_L3 = earthaccess.search_data(
    short_name="PACE_OCI_L3B_CHL",
    temporal=tspan,
    bounding_box=bbox
)
len(results_L3)


133

In [37]:
paths = earthaccess.open(results)
paths_L3 = earthaccess.open(results_L3)

QUEUEING TASKS | :   0%|          | 0/133 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/133 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/133 [00:00<?, ?it/s]

QUEUEING TASKS | :   0%|          | 0/60 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/60 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/60 [00:00<?, ?it/s]

In [40]:
# datatree = xr.open_datatree(paths_L3[0])
# dataset = xr.merge(datatree.to_dict().values()).chlor_a


In [45]:
dataset = xr.open_mfdataset(paths_L3,
    combine="nested",
    concat_dim="date",
)
